<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250527%5D%20FedAvg-MLP-LSTM%20with%2030%20CommRounds%20%26%205%20clients%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FedAvg-MLP-LSTM with 30 Communications Rounds & 5 clients on WSN-BFSF**

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [2]:
# ----------------------------
# Preprocessing and Partitioning
# ----------------------------
df = pd.read_csv("dataset.csv").dropna()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

X, y = df.drop('Class', axis=1), df['Class']
X = StandardScaler().fit_transform(X)
X, y = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [3]:
# ----------------------------
# Federated Setup
# ----------------------------
num_clients = 5
rounds = 30
client_X = np.array_split(X_train, num_clients)
client_y = np.array_split(y_train, num_clients)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test_cat = to_categorical(y_test)
num_classes = y_test_cat.shape[1]
input_shape = (X_test.shape[1], 1)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# ----------------------------
# Model Definition
# ----------------------------
def build_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, activation='tanh'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------------------------
# Training Loop
# ----------------------------
log = []

global_model = build_model(input_shape, num_classes)
global_weights = global_model.get_weights()

for r in range(rounds):
    round_start = time.time()
    client_weights = []
    for i in range(num_clients):
        local_model = build_model(input_shape, num_classes)
        local_model.set_weights(global_weights)
        X_c = client_X[i].reshape(client_X[i].shape[0], client_X[i].shape[1], 1)
        y_c = to_categorical(client_y[i], num_classes=num_classes)
        local_model.fit(X_c, y_c, epochs=1, batch_size=32, verbose=0)
        client_weights.append(local_model.get_weights())

    # FedAvg
    global_weights = [np.mean(np.array([w[i] for w in client_weights]), axis=0) for i in range(len(global_weights))]
    global_model.set_weights(global_weights)

    # Evaluation
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test_cat, axis=1)
    acc = accuracy_score(y_true, y_pred_classes)
    prec = precision_score(y_true, y_pred_classes, average='weighted')
    rec = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true, y_pred_classes)
    auc = roc_auc_score(y_test_cat, y_pred, multi_class='ovr')
    round_time = time.time() - round_start

    log.append({
        'Round': r+1,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc,
        'RoundTime(s)': round_time
    })

    print(f"Round {r+1}: Acc={acc:.8f}, F1={f1:.8f}, AUC={auc:.8f}, Time={round_time:.6f}s")
    print("Confusion Matrix:")
    print(cm)
    print(classification_report(y_true, y_pred_classes))

# ----------------------------
# Save Logs
# ----------------------------
df_log = pd.DataFrame(log)
df_log.to_csv("fedavg_training_log.csv", index=False)
print("Training log saved to fedavg_training_log.csv")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 37s 6ms/step
Round 1: Acc=0.67839700, F1=0.66797306, AUC=0.87276599, Time=322.842363s
Confusion Matrix:
[[33318  9952  9036   264]
 [17314 22942  6873  5441]
 [11107  2275 33823  5365]
 [    0     0     0 52571]]
              precision    recall  f1-score   support

           0       0.54      0.63      0.58     52570
           1       0.65      0.44      0.52     52570
           2       0.68      0.64      0.66     52570
           3       0.83      1.00      0.90     52571

    accuracy                           0.68    210281
   macro avg       0.67      0.68      0.67    210281
weighted avg       0.67      0.68      0.67    210281



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step
Round 2: Acc=0.87056367, F1=0.86646138, AUC=0.97629812, Time=322.737636s
Confusion Matrix:
[[34763 12986  4821     0]
 [ 1282 45720   184  5384]
 [   23   101 51291  1155]
 [    0     0  1282 51289]]
              precision    recall  f1-score   support

           0       0.96      0.66      0.78     52570
           1       0.78      0.87      0.82     52570
           2       0.89      0.98      0.93     52570
           3       0.89      0.98      0.93     52571

    accuracy                           0.87    210281
   macro avg       0.88      0.87      0.87    210281
weighted avg       0.88      0.87      0.87    210281



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

6572/6572 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step
Round 28: Acc=0.97176160, F1=0.97170329, AUC=0.99848899, Time=302.666139s
Confusion Matrix:
[[48516  2900  1154     0]
 [  434 52077    11    48]
 [   24    15 52531     0]
 [    0   804   548 51219]]
              precision    recall  f1-score   support

           0       0.99      0.92      0.96     52570
           1       0.93      0.99      0.96     52570
           2       0.97      1.00      0.98     52570
           3       1.00      0.97      0.99     52571

    accuracy                           0.97    210281
   macro avg       0.97      0.97      0.97    210281
weighted avg       0.97      0.97      0.97    210281



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step
Round 29: Acc=0.97383501, F1=0.97379822, AUC=0.99870037, Time=311.920333s
Confusion Matrix:
[[48892  2640  1038     0]
 [  487 52021    11    51]
 [   32    14 52524     0]
 [    0   826   403 51342]]
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     52570
           1       0.94      0.99      0.96     52570
           2       0.97      1.00      0.99     52570
           3       1.00      0.98      0.99     52571

    accuracy                           0.97    210281
   macro avg       0.97      0.97      0.97    210281
weighted avg       0.97      0.97      0.97    210281



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step
Round 30: Acc=0.97548043, F1=0.97542485, AUC=0.99898412, Time=300.871039s
Confusion Matrix:
[[48896  2717   957     0]
 [  355 52147    11    57]
 [   33    11 52526     0]
 [    0   563   452 51556]]
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     52570
           1       0.94      0.99      0.97     52570
           2       0.97      1.00      0.99     52570
           3       1.00      0.98      0.99     52571

    accuracy                           0.98    210281
   macro avg       0.98      0.98      0.98    210281
weighted avg       0.98      0.98      0.98    210281

Training log saved to fedavg_training_log.csv
